使用以下三阶多项式来生成和训练测试和训练数据的标签
$$
y = 5^{\frac{1}{x}} + 1.2x - 3.4\frac{x^2}{2!} + 5.6\frac{x^3}{3!} + \epsilon \quad \text{where} \quad \epsilon \sim \mathcal{N}(0, 0.1^2)
$$

In [61]:
import math
import numpy as np
import torch
from torch import nn
from train_ch3 import *
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cpu")

In [62]:
max_degree = 20  # 多项式的最大阶数
n_train, n_test = 100, 100  # 训练和测试数据集大小
true_w = np.zeros(max_degree)  # 分配大量的空间,这行代码创建了一个大小为 max_degree 的数组 true_w，初始值全部为 0。这个数组将存储生成数据时的“真实”多项式系数。
true_w[0:4] = np.array([5, 1.2, -3.4, 5.6])#这一行设置了“真实”多项式模型的系数。

features = np.random.normal(size=(n_train + n_test, 1))#这行代码从标准正态分布中随机生成 200 个输入特征，形状为 (200, 1)，作为模型的输入数据。
np.random.shuffle(features)#这行代码将生成的 features 数据进行随机打乱，保证数据是无序的，防止任何排序引入的偏差。
poly_features = np.power(features, np.arange(max_degree).reshape(1, -1))#这行代码生成多项式特征矩阵。np.arange(max_degree) 创建一个从 0 到 19 的数组，每个 features 中的元素被分别提升到这些指数的幂，
#从而生成多项式特征。例如，如果 features 中某个元素是 𝑥,则对应行会是 [1,𝑥,𝑥2,𝑥3,…,𝑥19]
np.random.shuffle(features)
for i in range(max_degree):#这段代码循环遍历每一列，将每列的值除以阶乘，math.gamma(i + 1) 表示 𝑖!（阶乘），这是因为生成的数据使用泰勒展开近似的多项式形式。这样可以将不同阶次的特征进行缩放，避免高阶项在数值上过大。
    poly_features[:, i] /= math.gamma(i + 1)  # gamma(n)=(n-1)!
# labels的维度:(n_train+n_test,)
labels = np.dot(poly_features, true_w)
labels += np.random.normal(scale=0.1, size=labels.shape)#这行代码为每个标签值加上一个从标准正态分布中抽样的噪声，噪声的标准差为 0.1。这是为了模拟真实数据中的噪声情况，使生成的标签数据不完全符合理想的多项式函数。

In [63]:
# NumPy ndarray转换为tensor
true_w, features, poly_features, labels = [torch.tensor(x, dtype=
    torch.float32) for x in [true_w, features, poly_features, labels]]

features[:2], poly_features[:2, :], labels[:2]

(tensor([[-0.1028],
         [-0.9377]]),
 tensor([[ 1.0000e+00, -1.1763e+00,  6.9184e-01, -2.7127e-01,  7.9773e-02,
          -1.8767e-02,  3.6793e-03, -6.1828e-04,  9.0910e-05, -1.1882e-05,
           1.3977e-06, -1.4946e-07,  1.4651e-08, -1.3257e-09,  1.1138e-10,
          -8.7347e-12,  6.4216e-13, -4.4434e-14,  2.9037e-15, -1.7977e-16],
         [ 1.0000e+00, -2.1724e+00,  2.3596e+00, -1.7087e+00,  9.2798e-01,
          -4.0319e-01,  1.4598e-01, -4.5304e-02,  1.2302e-02, -2.9694e-03,
           6.4508e-04, -1.2740e-04,  2.3063e-05, -3.8540e-06,  5.9802e-07,
          -8.6609e-08,  1.1759e-08, -1.5027e-09,  1.8136e-10, -2.0736e-11]]),
 tensor([ -0.2782, -15.2811]))

对模型进行测试和评估

In [64]:
def evaluate_loss(net, data_iter, loss):  #@save
    """评估给定数据集上模型的损失"""
    metric = d2l.Accumulator(2)  # 损失的总和,样本数量
    for X, y in data_iter:
        out = net(X)
        y = y.reshape(out.shape)
        l = loss(out, y)
        metric.add(l.sum(), l.numel())
    return metric[0] / metric[1]

In [65]:
class Net(nn.Module):
    def __init__(self, input_shape, *args, **kwargs ) -> None:
        super().__init__(*args, **kwargs)
        self.fl = nn.Sequential(nn.Linear(input_shape, 1, bias=False))
    
    def forward(self, x):
        x = self.fl(x)
        return x

In [66]:
writer = SummaryWriter("log")
def train(train_features, test_features, train_labels, test_labels,
          num_epochs=400):
    loss = nn.MSELoss(reduction='none')
    loss = loss.to(device)
    input_shape = train_features.shape[-1]
    # 不设置偏置，因为我们已经在多项式中实现了它
    net = Net(input_shape).to(device)
    batch_size = min(10, train_labels.shape[0])
    train_iter = d2l.load_array((train_features, train_labels.reshape(-1,1)),batch_size)
    test_iter = d2l.load_array((test_features, test_labels.reshape(-1,1)),batch_size, is_train=False)
    trainer = torch.optim.SGD(net.parameters(), lr=0.01)
    for epoch in range(num_epochs):
        for epoch in range(num_epochs):
            #训练
            #print("----------第{}轮训练开始----------".format(epoch + 1))
            net.train()
            for data in train_iter:
                X , y = data
                X = X.to(device)
                y = y.to(device)
                y_hat = net(X)
                l = loss(y_hat , y)
                trainer.zero_grad()
                l.mean().backward()
                trainer.step()
            #测试
            net.eval()
            with torch.no_grad():
                for data in  test_iter:
                    X , y = data
                    X = X.to(device)
                    y = y.to(device)
                    y_hat = net(X)
                    l = loss(y_hat , y)
        if epoch == 0 or (epoch + 1) % 20 == 0:
            writer.add_scalar("train_loss1",evaluate_loss(net, train_iter, loss),epoch + 1)
            writer.add_scalar("test_loss1",evaluate_loss(net, test_iter, loss),epoch + 1)
    print('weight:', net[0].weight.data.numpy())

In [67]:
train(poly_features[:n_train, :4], poly_features[n_train:, :4],
      labels[:n_train], labels[n_train:])

TypeError: 'Net' object is not subscriptable